In [ ]:
# Проект Дашборд конверсий

In [ ]:
## Загрузка библиотек

In [2]:
import pandas as pd
import requests as req

In [3]:
## Чтение и выборка зарегистрированных и посещаемых пользователей

In [4]:
START = '2023-03-01'
END = '2023-09-01'
visitors_req = req.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01')
visitors = pd.DataFrame(visitors_req.json())
regs_req = req.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01')
regs = pd.DataFrame(regs_req.json())

In [ ]:
## Анализ зарегистрированных пользователей

In [6]:
visitors.describe()

,visit_id,platform,user_agent,datetime
count,263459,263459,263459,263459
unique,146085,4,32,261204
top,2cee521c-1747-4457-9e07-b186d6fdee05,web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...,2023-06-01T19:26:40
freq,4,236301,13623,3


In [ ]:
## Анализ посещаемых пользователей

In [5]:
regs.describe()

,datetime,user_id,email,platform,registration_type
count,21836,21836,21836,21836,21836
unique,21821,21836,20868,3,4
top,2023-04-30T10:23:02,fe03e08a-8e86-4b5f-9a50-c18a02b0d234,ujones@example.com,android,email
freq,2,1,6,10582,8996
